# Assignment 5: Classification

## Step 1. Assumptions

* Divide the data by z-layer into 11 slices.  
* Assume the mean density of synapses in a slice can be classified into 2 groups: $W_i$.  
* $W_i = \{\text{High density},\text{  Low density}\}$  




## Step 2. Define classification problem

* We then randomly choose a small grid, and use the $X$ and $Y$ position to predict this grid belongs to high or low density.  
  
* $N$ is the number of synapses (observations)
* $X_i = X\text{ position} $  
* $Y_i = Y\text{ position} $  
* $H_0 = N \perp\!\!\!\perp X, Y$ positions
* $H_1 = N \text{ is not} \perp\!\!\!\perp X, Y$ positions  


* Becuase we will do the classification, the objective is to minimize the expected error:  
* $E[l] = \sum_{n=1}^{\infty}I(\hat{W_i} \neq W_i)$ where $I$ is the indicator function.


## Step 3. Provide algorithms

Classification methods:

* lda (Linear Discriminant Analysis): No parameter.
* qda (Quadratic Discriminant Analysis): No parameter.
* svm (Support Vector Machine): Penalty parameters set to 0.5, the default value.
* knn (K-Nearest Neighbours): Number of neighbors set to 3, the default value.
* rf (Random Forest): Use default value.

## Step 4. Sample data from null and alternative model

## Step 5. Compute accuracy

## Step 6. Apply on data

## Step 7. Interpret